# Clustering and Maps
## Importing libraries

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import numpy as np

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


- ### Reading data from previous notebook
- ### Selecting data points containing the word 'Toronto'

In [2]:
toronto = pd.read_csv('toronto.csv')
toronto = toronto[toronto['Borough'].str.contains("Toronto")]
toronto.shape

(38, 6)

## Finding the coordinates for Toronto

In [3]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Creating map by using "Folium"

In [4]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Clustering

In [5]:
# set number of clusters
kclusters = 4

toronto_clustering = toronto.drop('Postcode', 1)
toronto_clustering = toronto_clustering.drop('Neighbourhood', 1)
toronto_clustering = toronto_clustering.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [6]:
# add clustering labels

toronto_clustering=toronto
toronto_clustering['Cluster Labels'] = kmeans.labels_

toronto_clustering.head() # check the last columns!

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
37,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
41,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0
42,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0
43,43,M4M,East Toronto,Studio District,43.659526,-79.340923,0
44,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0


## Plotting Clusters

In [7]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clustering['Latitude'], toronto_clustering['Longitude'], toronto_clustering['Neighbourhood'], toronto_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters